## Instalando e Carregando os Pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [ ]:
!pip install -q torch==2.0.0 transformers==4.28.1 watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# Imports
import csv
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "version 0" --iversions

Author: version 0

transformers: 4.28.1
csv         : 1.0
torch       : 2.0.0



## Carregando os Dados

In [ ]:
# Lista de frases
frases = [
    "Eu amei este filme!",
    "Este produto é terrível.",
    "A comida estava deliciosa.",
    "O serviço foi péssimo.",
    "Este livro é incrível.",
    "A história da Inglaterra é curiosa.",
    "Não gostei daquele novo modelo de carro.",
    "Aqueles foi um dos melhores jogos de futebol da história.",
    "Este curso é fantástico e de alto nível.",
    "O serviço do garçom não esteve à altura do nome do restaurante."
]


In [ ]:
# Imprime a lista de frases
print(frases)

['Eu amei este filme!', 'Este produto é terrível.', 'A comida estava deliciosa.', 'O serviço foi péssimo.', 'Este livro é incrível.', 'A história da Inglaterra é curiosa.', 'Não gostei daquele novo modelo de carro.', 'Aqueles foi um dos melhores jogos de futebol da história.', 'Este curso é fantástico e de alto nível.', 'O serviço do garçom não esteve à altura do nome do restaurante.']


In [ ]:
# Nome do objeto
texts = frases

In [ ]:
# 1: positivo, 0: negativo
labels = [1, 0, 1, 0, 1, 1, 0, 1, 1, 0]

## Pré-Processamento

In [ ]:
RANDOM_SEED = 42

In [ ]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size = 0.2,
                                                                      random_state = RANDOM_SEED)

In [ ]:
# Carrega o tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Classe de tokenização dos dados
class SentimentAnalysisTokenizer(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):

        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer.encode_plus(text,
                                            add_special_tokens = True,
                                            max_length = self.max_length,
                                            padding = 'max_length',
                                            truncation = True,
                                            return_tensors = 'pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label)
        }

In [ ]:
MAX_LENGTH = 64

In [ ]:
# Aplica a tokenização
train_dataset = SentimentAnalysisTokenizer(train_texts, train_labels, tokenizer, MAX_LENGTH)

In [ ]:
# Aplica a tokenização
test_dataset = SentimentAnalysisTokenizer(test_texts, test_labels, tokenizer, MAX_LENGTH)

In [ ]:
# Cria o data loader
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)

In [ ]:
# Cria o data loader
test_loader = DataLoader(test_dataset, batch_size = 16)

## Loop de Treino, Avaliação e Inferência

In [ ]:
# Função para treinar o modelo
def train_epoch(model, data_loader, criterion, optimizer, device):

    model.train()
    total_loss = 0

    for batch in data_loader:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

In [ ]:
# Função para avaliar o modelo
def eval_epoch(model, data_loader, criterion, device):

    model.eval()
    total_loss = 0

    with torch.no_grad():

        for batch in data_loader:

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [ ]:
# Função para obter previsões do modelo
def predict(model, data_loader, device):

    model.eval()
    predictions = []

    with torch.no_grad():

        for batch in data_loader:

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.tolist())

    return predictions

## Construção do Modelo

In [ ]:
# Inicialização do modelo e configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Carrega o modelo
modelo = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Hiperparâmetros
EPOCHS = 10
LEARNING_RATE = 2e-5

In [ ]:
# Otimizador
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)

**torch.optim.AdamW()** é uma classe de otimizador no PyTorch que implementa a versão AdamW do algoritmo de otimização Adam. AdamW é uma extensão do otimizador Adam que inclui correções de decaimento de peso (Weight Decay) para melhorar a generalização do modelo em tarefas de aprendizado profundo.

Aqui está uma descrição dos principais parâmetros da classe torch.optim.AdamW:

- params (iterável): Um iterável de tensores (geralmente os parâmetros do modelo) que serão otimizados.


- lr (float, opcional): Taxa de aprendizado (learning rate), um hiperparâmetro que controla o tamanho do passo de atualização dos parâmetros do modelo. O valor padrão é 1e-3 (0.001).


- betas (par de floats, opcional): Coeficientes usados para calcular as médias móveis exponenciais dos gradientes e seus quadrados. Os valores padrão são (0.9, 0.999).


- eps (float, opcional): Termo adicionado para melhorar a estabilidade numérica durante a otimização. O valor padrão é 1e-8.


-  weight_decay (float, opcional): Coeficiente de decaimento de peso (Weight Decay), um hiperparâmetro que penaliza os parâmetros do modelo para evitar overfitting. O valor padrão é 0.


- amsgrad (bool, opcional): Se True, usa a variante AMSGrad do otimizador Adam, que é mais robusta em certos casos, mas geralmente não é necessário. O valor padrão é False.

In [ ]:
# Função de perda
criterion = torch.nn.CrossEntropyLoss()

**torch.nn.CrossEntropyLoss()** é uma classe de função de perda (loss function) no PyTorch, usada para tarefas de classificação multi-classe. A função de perda de entropia cruzada combina a função LogSoftmax e a função de perda NLLLoss (Negative Log-Likelihood Loss) em uma única classe, tornando-a conveniente e eficiente para uso em problemas de classificação.

A entropia cruzada mede a diferença entre duas distribuições de probabilidade, neste caso, a distribuição prevista pelo modelo e a distribuição real (rótulos verdadeiros). Durante o treinamento, o objetivo é minimizar a perda de entropia cruzada, o que leva a um melhor ajuste do modelo aos dados.

## Treinamento e Avaliação do Modelo

In [ ]:
# Treinamento e validação do modelo
for epoch in range(EPOCHS):
    train_loss = train_epoch(modelo, train_loader, criterion, optimizer, device)
    test_loss = eval_epoch(modelo, test_loader, criterion, device)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 0.6756783723831177, Test Loss: 0.7047009468078613
Epoch 2/10, Train Loss: 0.662405252456665, Test Loss: 0.7062666416168213
Epoch 3/10, Train Loss: 0.6563072204589844, Test Loss: 0.7114291191101074
Epoch 4/10, Train Loss: 0.6199197769165039, Test Loss: 0.7192910313606262
Epoch 5/10, Train Loss: 0.6026990413665771, Test Loss: 0.7265456318855286
Epoch 6/10, Train Loss: 0.5918408632278442, Test Loss: 0.7393703460693359
Epoch 7/10, Train Loss: 0.5420755743980408, Test Loss: 0.7633539438247681
Epoch 8/10, Train Loss: 0.4731328785419464, Test Loss: 0.7956526875495911
Epoch 9/10, Train Loss: 0.5267807245254517, Test Loss: 0.8385111093521118
Epoch 10/10, Train Loss: 0.48884981870651245, Test Loss: 0.8857986330986023


In [ ]:
# Salva o modelo em disco
torch.save(modelo, 'modelo_mp1.pt')

In [ ]:
# Carrega o modelo do disco
modelo_final = torch.load('modelo_mp1.pt')

## Testando o Modelo com Novos Dados

In [ ]:
# Novos dados
novas_frases = ['Eu gostei muito deste filme.',
                'O atendimento do restaurante foi decepcionante.']

In [ ]:
# Aplica a tokenização
dataset = SentimentAnalysisTokenizer(novas_frases, [0] * len(novas_frases), tokenizer, MAX_LENGTH)

In [ ]:
# Cria o data loader
loader = DataLoader(dataset, batch_size = 16)

In [ ]:
# Previsões
previsoes = predict(modelo_final, loader, device)

In [ ]:
# Análise de sentimento
for text, prediction in zip(novas_frases, previsoes):
    print(f'Sentença: {text} | Sentimento: {"positivo" if prediction else "negativo"}')

Sentença: Eu gostei muito deste filme. | Sentimento: positivo
Sentença: O atendimento do restaurante foi decepcionante. | Sentimento: positivo
